In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pwd

/kaggle/working


In [2]:
!pip install lightning -q

In [3]:
import lightning as L

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [6]:
class bart_model(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained("t5-base")
    def forward(self, batch_input):
        output = self.model(**batch_input)
        return output.loss, output.logits
    def training_step(self, batch, batch_idx):
        loss, output = self(batch)
        self.log('train_loss',loss, prog_bar=True, logger=True)
        print('train_loss:', loss)
        return loss
    def validation_step(self, batch, batch_idx):
        loss, output = self(batch)
        self.log('val_loss',loss, prog_bar=True, logger=True)
        print('val_lss:', loss)
        return loss
    def test_step(self, batch, batch_idx):
        loss, output = self(batch)
        self.log('test_loss',loss, prog_bar=True, logger=True)
        return loss
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=2e-4)

In [10]:
from transformers import T5ForConditionalGeneration
model = bart_model()

In [13]:
import torch
checkpoint = torch.load("/kaggle/input/t5-v123/lightning_logs/version_0/checkpoints/epoch=0-step=4119.ckpt", map_location=torch.device('cpu'))

In [16]:
state_dict = checkpoint['state_dict']

In [17]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [19]:
from transformers import T5Tokenizer

In [21]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")

In [26]:
question = 'what is color'
context = "Color is a perceptual property of light that is typically associated with the visual experience of humans and many other animals. It is a result of the interaction between light, objects, and the human visual system. Light is made up of electromagnetic waves that travel in different wavelengths. The different wavelengths of light correspond to different colors. When light strikes an object, some wavelengths are absorbed by the object, while others are reflected or transmitted. The wavelengths that are reflected or transmitted are detected by the human eye, which contains specialized cells called cones that are sensitive to different ranges of wavelengths. The human eye has three types of cones: those that are most sensitive to short wavelengths (which we perceive as blue), those sensitive to medium wavelengths (perceived as green), and those sensitive to long wavelengths (perceived as red). The combination and intensity of signals from these cones allow us to perceive a wide range of colors. Color perception can also be influenced by factors such as lighting conditions, the surrounding environment, and individual differences in color vision. Additionally, color is subjective and can evoke different emotions and associations in different individuals and cultures. In summary, color is a visual sensation that arises from the interaction of light, objects, and the human visual system, allowing us to perceive and differentiate various hues and shades."
inputs = tokenizer.encode_plus(question, context, max_length=256, padding='max_length', 
                            truncation=True, return_tensors='pt').input_ids

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [29]:
outputs = model.model.generate(inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

the color of light is a combination of the wavelengths of light and the wavelengths of
